## Setup

In [1]:
import ipywidgets as widgets
from ipywidgets import Button, Layout, IntSlider, HBox, VBox, interact, interact_manual
import os
import csv
from IPython.display import display, clear_output, Markdown, Image
import rpy2.rinterface
%load_ext rpy2.ipython

In [2]:
## Initializing all relevant variables as global variables
global sample_column_id
global graph_output_dir
global stats_output_dir
global amr_count_matrix_filepath
global amr_metadata_filepath
global megares_annotation_filename
global biom_file
global tre_file
global tax_fasta
global taxa_file
global microbiome_temp_metadata_file
global list_vals_a
global list_vals_m

### Text Entry Widgets

In [3]:
sample_column_id = widgets.Text(
    value='ID',
    placeholder='What is the name of the column in the metadata file where the sample IDs are stored? ',
    description='Column Name:',
    disabled=False,
    layout=Layout(width='70%'),
    style = {'description_width': 'initial'}
)

graph_output_dir = widgets.Text(
    value='graphs',
    placeholder='Name the output directory for where the graphs should be saved',
    description='Directory Name for Graphs: ',
    disabled=False,
    layout=Layout(width='70%'),
    style = {'description_width': 'initial'}
)

stats_output_dir = widgets.Text(
    value='stats',
    placeholder='Name the output directory for where the statistics should be saved',
    description='Directory Name for Stats: ',
    disabled=False,
    layout=Layout(width='70%'),
    style = {'description_width': 'initial'}
)

direct_name_data = VBox([sample_column_id, graph_output_dir, stats_output_dir])

## File Paths
Creating elements to enter in file names in text boxes

In [4]:
amr_count_file = widgets.Dropdown(
    options=os.listdir(str(os.getcwd()) + "/data"),
    value = "test_AMR_analytic_matrix.csv",
    description='AMR Count Matrix File:',
    disabled=False,
    layout=Layout(width='50%'),
    style = {'description_width': 'initial'}
)

amr_metadata_file = widgets.Dropdown(
    options=os.listdir(str(os.getcwd()) + "/data"),
    value = "test_AMR_metadata.csv",
    description='Metadata File:',
    disabled=False,
    layout=Layout(width='50%'),
    style = {'description_width': 'initial'}
)

megares_annot = widgets.Dropdown(
    options=os.listdir(str(os.getcwd()) + "/data"),
    value = "test_AMR_megares_annotations_v1.03.csv",
    description='MEGARes Annotation File:',
    disabled=False,
    layout=Layout(width='50%'),
    style = {'description_width': 'initial'}
)

resistome_filenames = VBox([amr_count_file, amr_metadata_file, megares_annot])

In [5]:
biom = widgets.Dropdown(
    options=os.listdir(str(os.getcwd()) + "/data"),
    value = "test_16S_otu_table_json.biom",
    description='Biom File:',
    disabled=False,
    layout=Layout(width='50%'),
    style = {'description_width': 'initial'}
)

tre = widgets.Dropdown(
    options=os.listdir(str(os.getcwd()) + "/data"),
    value = "test_16S_tree.nwk",
    description='tree file:',
    disabled=False,
    layout=Layout(width='50%'),
    style = {'description_width': 'initial'}
)

tax = widgets.Dropdown(
    options=os.listdir(str(os.getcwd()) + "/data"),
    value = "test_16S_dna-sequences.fasta",
    description='fasta file:',
    disabled=False,
    layout=Layout(width='50%'),
    style = {'description_width': 'initial'}
)

taxa = widgets.Dropdown(
    options=os.listdir(str(os.getcwd()) + "/data"),
    value = "test_16S_taxonomy.tsv",
    description='taxonomy file:',
    disabled=False,
    layout=Layout(width='50%'),
    style = {'description_width': 'initial'}
)

microbiome_temp = widgets.Dropdown(
    options=os.listdir(str(os.getcwd()) + "/data"),
    value = "test_16S_metadata.csv",
    description='microbiome temp metadata file:',
    disabled=False,
    layout=Layout(width='50%'),
    style = {'description_width': 'initial'}
)

microbiome_filenames = VBox([biom, tre, tax, taxa, microbiome_temp])

In [6]:
## AMR Text Boxes

microbiome_filenames = VBox([biom, tre, tax, taxa, microbiome_temp])

In [7]:
## Function to save all the filepaths to the normal variable expected by the rest of the code
def save_filepaths(b):
    
    global amr_count_matrix_filepath
    global amr_metadata_filepath
    global megares_annotation_filename
    
    global biom_file
    global tre_file
    global tax_fasta
    global taxa_file
    global microbiome_temp_metadata_file
    
    amr_count_matrix_filepath = "data/" + str(amr_count_file.value)
    amr_metadata_filepath = "data/" + str(amr_metadata_file.value)
    megares_annotation_filename = "data/" + str(megares_annot.value)
    
    biom_file = "data/" + str(biom.value)
    tre_file = "data/" + str(tre.value)
    tax_fasta = "data/" + str(tax.value)
    taxa_file = "data/" + str(taxa.value)
    microbiome_temp_metadata_file = "data/" + str(microbiome_temp.value)
    
    print("AMR Count Matrix Filepath: " + str(amr_count_file.value))
    print("AMR Metadata Filepath: " + str(amr_metadata_file.value))
    print("MEGARes Annotation Filepath: " + str(megares_annot.value))
    print("Biom Filepath: " + str(biom.value))
    print("Fasta Filepath: " + str(tre.value))
    print("Taxonomy Filepath: " + str(taxa.value))
    print("Microbiome Temp Metadata Filepath: " + str(microbiome_temp.value))
    print()
    print("All filepaths saved")
    


In [8]:
save_filepath_button = widgets.Button(
    description="Save the filepaths for analysis", 
    icon = 'save',
    layout=Layout(width='70%'))

save_filepath_button.on_click(save_filepaths)

## AMR Exploratory variables
Multiple text input boxes backend code

In [9]:
### Making the function for the slider to choose number of accordions 
exp_graph_var_amr = widgets.IntSlider( 
    value=5,
    min=0,
    max=10,
    step=1,
    description='AMR',
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

exp_graph_var_microbiome = widgets.IntSlider( 
    value=5,
    min=0,
    max=10,
    step=1,
    description= 'Microbiome',
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

In [10]:
### Making the different text boxes for the 4 variables
name = widgets.Text(
    value='',
    placeholder='Name',
    disabled=False,
    layout=Layout(width='70%'),
    style = {'description_width': 'initial'}
)


#widgets.Text(value='', placeholder='Name', disabled=False, layout=Layout(width='70%'), style = {'description_width': 'initial'})


subsets = widgets.Text(
    value='',
    placeholder='Subsets',
    disabled=False,
    layout=Layout(width='70%'),
    style = {'description_width': 'initial'}
)

exploratory_var = widgets.Text(
    value='',
    placeholder='Exploratory Variable',
    disabled=False,
    layout=Layout(width='70%'),
    style = {'description_width': 'initial'}
)

order = widgets.Text(
    value='',
    placeholder='Order',
    disabled=False,
    layout=Layout(width='70%'),
    style = {'description_width': 'initial'}
)


In [11]:
### Creating a vertical box to store all of the text boxes into a single object 
menu1 = widgets.VBox([name, subsets, exploratory_var, order])

In [12]:
### Updating the number of accordion pages based on the value selected in the slider above, then prints it
def graph_vars_amr(exp_graph_var):
    list_widgets_a = []
    
    # This creates all the new variables to store the text boxes in
    for i in range(exp_graph_var):
        
        name_box = "global name_a{}; name_a{} = widgets.Text(placeholder='name',layout=Layout(width='70%'))".format(i,i)
        subset_box = "global subset_a{}; subset_a{} = widgets.Text(placeholder='Subset',layout=Layout(width='70%'))".format(i,i)
        exp_box = "global exploratory_a{}; exploratory_a{} = widgets.Text(placeholder='Exploratory Variable',layout=Layout(width='70%'))".format(i,i)
        order_box = "global order_a{}; order_a{} = widgets.Text(placeholder='Order',layout=Layout(width='70%'))".format(i,i)
        exec(name_box)
        exec(subset_box)
        exec(exp_box)
        exec(order_box)
             
    
    # This will assign a new menu_ variable for each accordion that needs to be printed
    for i in range(exp_graph_var):
        string = "menu_a{} = widgets.VBox([name_a{}, subset_a{}, exploratory_a{}, order_a{}])".format(i, i, i, i, i)
        exec(string)

    
    # This should append all the menu variables into the list_widgest list and pass it into the accordion widget
    for i in range(0, exp_graph_var):
        string = "list_widgets_a.append(menu_a{})".format(i)
        exec(string)
    
    # Creates and displays the final accordion widget
    accordion_a = widgets.Accordion(children=list_widgets_a)
    return accordion_a


def graph_vars_mic(exp_graph_var):
    list_widgets_m = []
    # This creates all the new variables to store the text boxes in
    for i in range(exp_graph_var):
        
        name_box_m = "global name_m{}; name_m{} = widgets.Text(placeholder='name',layout=Layout(width='70%'))".format(i,i)
        subset_box_m = "global subset_m{}; subset_m{} = widgets.Text(placeholder='Subset',layout=Layout(width='70%'))".format(i,i)
        exp_box_m = "global exploratory_m{}; exploratory_m{} = widgets.Text(placeholder='Exploratory Variable',layout=Layout(width='70%'))".format(i,i)
        order_box_m = "global order_m{}; order_m{} = widgets.Text(placeholder='Order',layout=Layout(width='70%'))".format(i,i)
        exec(name_box_m)
        exec(subset_box_m)
        exec(exp_box_m)
        exec(order_box_m)
             
    
    # This will assign a new menu_ variable for each accordion that needs to be printed
    for i in range(exp_graph_var):
        string = "menu_m{} = widgets.VBox([name_m{}, subset_m{}, exploratory_m{}, order_m{}])".format(i, i, i, i, i)
        exec(string)
    
    # This should append all the menu variables into the list_widgest list and pass it into the accordion widget
    for i in range(0, exp_graph_var):
        string = "list_widgets_m.append(menu_m{})".format(i)
        exec(string)
    
    # Creates and displays the final accordion widget
    accordion_m = widgets.Accordion(children=list_widgets_m)
    return accordion_m


In [13]:
## Store all the variables in amr_exp in order to be written out to a csv file
global list_vals_a
global list_vals_m
list_vals_a = []
list_vals_m = []

def save_print_variables(amr, mic):
    list_vals_a = []
    list_vals_m = []
    
    exp = ["_a", "_m"]
    num = [amr, mic]
    
    for i in range(2):
        for j in range(num[i]):
            analysis = exp[i]
            exec("order_new{}{} = order_format(order{}{}.value)".format(analysis, j, analysis, j))
            exec("subset_new{}{} = subset_format(subset{}{}.value)".format(analysis, j, analysis, j))
            string = 'list_vals{}.append([name{}{}.value, subset_new{}{}, exploratory{}{}.value, order_new{}{}])'.format(analysis, analysis, j, analysis, j, analysis, j, analysis, j)
            exec(string)

    #print(list_vals_a)
    #print("")
    #print(list_vals_m)
    
    return list_vals_a, list_vals_m

In [15]:
# Create button to save variables from exploratory variables
# list_vals_a, list_vals_m = save_print_variables(exp_graph_var_amr.value, exp_graph_var_microbiome.value)

def save_explore_vars(b):
    list_vals_a, list_vals_m = save_print_variables(exp_graph_var_amr.value, exp_graph_var_microbiome.value)
    print("Exploratory variables saved")

save_print_vars_button = widgets.Button(
    description="Save the exploratory variables for analysis", 
    icon = 'save',
    layout=Layout(width='70%'))

save_print_vars_button.on_click(save_explore_vars)

# display(save_print_vars_button)



NameError: name 'order_format' is not defined

In [ ]:
## Create the tabs to enter in variable data.
def var_info(amr, mic):
    tab = widgets.Tab()
    if mic == 0:
        tab_contents = ["AMR"]
        children = [graph_vars_amr(amr)]
        tab.set_title(0, "AMR")
    elif amr == 0:
        tab_contents = ["Microbiome"]
        children = [graph_vars_mic(mic)]
        tab.set_title(0, "Microbiome")
    else: 
        tab_contents = ["AMR", "Microbiome"]
        children = [graph_vars_amr(amr), graph_vars_mic(mic)]
        tab.set_title(0, "AMR")
        tab.set_title(1, "Microbiome")
    
    tab.children = children
    tab.titles = tab_contents
    return tab

In [ ]:
# Reformats the order variable
def order_format(order_og):
    # Splits the text into the different items in the list
    order_list = order_og.split(",")
    
    # Removes the white spaces before and after each item
    order_list = [item.strip() for item in order_list]

    # Adds the quotation marks around each item
    order_list = ['"' + item + '"' for item in order_list]
    
    # Removing unnecessary characters from string
    order_list = 'c({})'.format(order_list)
    order_list = order_list.replace("[", "")
    order_list = order_list.replace("]", "")
    order_list = order_list.replace("'", "")
    order_list = order_list.replace(", ", ",")
    
    if order_og == "":
        order_list = ""
    
    return order_list

# Reformats the subset variable
def subset_format(subset_og):
    # Splits the text into the different items in the list
    order_list = subset_og.split(",")
    
    # Removes the white spaces before and after each item
    order_list = [item.strip() for item in order_list]

    # Adds the quotation marks around each item
    order_list = ["'" + item + "'" for item in order_list]
    
    # Removing unnecessary characters from string
    order_list = 'list({})'.format(order_list)
    order_list = order_list.replace("[", "")
    order_list = order_list.replace("]", "")
    order_list = order_list.replace('"', "")
    order_list = order_list.replace(", ", ",")
    
    if subset_og == "":
        order_list = ""
    
    
    return order_list


In [ ]:
# Function to save all variables into .csv file
def vars_to_csv(b):
    with open('metagenome_analysis_vars.csv','w', newline='') as f:  
        writer = csv.writer(f)
        # Column names
        writer.writerow(['sample_column_id', sample_column_id.value])
        writer.writerow(['graph_output_dir', graph_output_dir.value])
        writer.writerow(['stats_output_dir', stats_output_dir.value])

        # Filepaths
        writer.writerow(['amr_count_matrix_filepath', amr_count_matrix_filepath])
        writer.writerow(['amr_metadata_filepath', amr_metadata_filepath])
        writer.writerow(['megares_annotation_filename', megares_annotation_filename])
        writer.writerow(['biom_file', biom_file])
        writer.writerow(['tre_file', tre_file])
        writer.writerow(['tax_fasta', tax_fasta])
        writer.writerow(['taxa_file', taxa_file])
        writer.writerow(['microbiome_temp_metadata_file', microbiome_temp_metadata_file])

        # AMR exploratory variables
        writer.writerow(["AMR_exploratory_analyses"])
        writer.writerows(list_vals_a)

        # Microbiome exploratory variables
        writer.writerow(["microbiome_exploratory_analyses"])
        writer.writerows(list_vals_m)
    print("Variables Exported. Check directory for .csv file")

In [ ]:
vars_save_button = widgets.Button(
    description="Save variables for analysis script", 
    icon = 'save',
    layout=Layout(width='50%'))

vars_save_button.on_click(vars_to_csv)

## R Code

In [3]:
def stage_script(b):
    %R -o amr_melted_analytic -o microbiome_melted_analytic -o amr_melted_raw_analytic -o microbiome_melted_raw_analytic
    %R source("staging_script.R")

stage_script_button = widgets.Button(
    description="Run the Staging Script to initialize objects", 
    icon = 'gear',
    layout=Layout(width='50%'))

stage_script_button.on_click(stage_script)

# display(stage_script_button)

Button(description='Run the Staging Script to initialize objects', icon='gear', layout=Layout(width='50%'), st…

In [ ]:
## For viewing the various metadata files generated from the script
out = widgets.Output(layout={'border': '1px solid black'})
file_open = widgets.Dropdown(
    options=['amr_melted_analytic', 'amr_melted_raw_analytic', 'microbiome_melted_analytic', 'microbiome_melted_raw_analytic'],
    description='file:',
    disabled=False,
)

def open_file(b):
    global df
    if file_open.value == "amr_melted_analytic":
        print("amr_melted_analytic")
        df = amr_melted_analytic
        display(df)        
        display(out)
    elif file_open.value == "amr_melted_raw_analytic":
        print("amr_melted_raw_analytic")
        display(amr_melted_raw_analytic)
        df = amr_melted_raw_analytic
        display(out)
    elif file_open.value == "microbiome_melted_analytic":
        print("microbiome_melted_analytic")
        display(microbiome_melted_analytic)
        df = microbiome_melted_analytic
        display(out)
    elif file_open.value == "microbiome_melted_raw_analytic":
        print("microbiome_melted_raw_analytic")
        display(microbiome_melted_raw_analytic)
        df = microbiome_melted_raw_analytic
        display(out)

open_file_button = widgets.Button(
    description="Open File", 
    icon = 'open',
    layout=Layout(width='70%'))

open_file_button.on_click(open_file)

## Filter metadata files based on level_ID

In [ ]:
def filter_level_ID_fun(b):
    global class_df
    global a_select
    global df
    
    if file_open.value == "amr_melted_analytic":
        df = amr_melted_analytic      
    elif file_open.value == "amr_melted_raw_analytic":
        df = amr_melted_raw_analytic
    elif file_open.value == "microbiome_melted_analytic":
        df = microbiome_melted_analytic
    elif file_open.value == "microbiome_melted_raw_analytic":
        df = microbiome_melted_raw_analytic
    
    items = ['All']+sorted(df['Level_ID'].unique().tolist())

    a_slider = widgets.IntSlider(min=0, max=len(df), step=1, value=10000)
    a_select =  widgets.Select(options=items)

    def update_select_range(*args):
        if a_select.value=='All':
            a_slider.max = len(df)
        else:
            a_slider.max = len(df[df['Level_ID']==a_select.value])

    a_select.observe(update_select_range, 'value')

    def view1(Level_ID='',Count=3):
        if Level_ID=='All': return df.head(Count)
        return df[df['Level_ID']==Level_ID].head(Count)

    print(file_open.value)
    widgets.interact(view1,Count=a_slider,Level_ID=a_select)

open_file_filter_button = widgets.Button(
    description="Open File", 
    icon = 'open',
    layout=Layout(width='50%'))

open_file_filter_button.on_click(filter_level_ID_fun)

## Print Figure

In [1]:
def print_fig(b):
    global class_df
    
    if a_select.value == "All":
        print("Level_ID cannot be 'All'. Please select a different Level_ID.")
    else:
        class_df = df[df['Level_ID']== str(a_select.value)]
        import plotly.express as px
        #df.plot(kind='bar', x='ID', y='Normalized_Count',fill='Name', mode='stack')
        display(px.bar(x=class_df['ID'], y=class_df['Normalized_Count'] , color=class_df['Name'])) # hover text goes here

print_fig_button = widgets.Button(
    description="Print Plot", 
    icon = 'open',
    layout=Layout(width='50%'))

print_fig_button.on_click(print_fig)

NameError: name 'widgets' is not defined